## Crearea unui model word2vec antrenat pe intregul set de date

In [1]:
import import_ipynb
from FunctionsList import *
import pandas as pd
from gensim.models import Word2Vec
from gensim import models

importing Jupyter notebook from FunctionsList.ipynb


In [2]:
data = pd.read_csv('Data/OriginalDataset.csv')
data = data['review_content']

In [3]:
data.head()

0        WOW! Fără cuvinte… pur și simplu excepțional.
1    Incep prin a spune ca am precomandat telefonul...
2            Sunt extraordinar de mulțumit de telefon.
3    Intradevar este un telefon foarte scump, dar c...
4    Detin un 13 pro max sierra 1tb din septembrie ...
Name: review_content, dtype: object

In [4]:
data = data.dropna()
data

0            WOW! Fără cuvinte… pur și simplu excepțional.
1        Incep prin a spune ca am precomandat telefonul...
2                Sunt extraordinar de mulțumit de telefon.
3        Intradevar este un telefon foarte scump, dar c...
4        Detin un 13 pro max sierra 1tb din septembrie ...
                               ...                        
41923    Un telefon foarte bun, se mișcă foarte bine pe...
41924    Păcat, că e un telefon frumos dar se incinge f...
41925                                Un telefon foarte bun
41926    Elegant, fiabil, frumos, functional si eficien...
41927    PRO - ecran luminos, durata bateriei excelentă...
Name: review_content, Length: 41927, dtype: object

In [5]:
# Curata setul de date 
#lowercase
data = data.apply(lambda x: lowercase_text(x))
data.head()

0        wow! fără cuvinte… pur și simplu excepțional.
1    incep prin a spune ca am precomandat telefonul...
2            sunt extraordinar de mulțumit de telefon.
3    intradevar este un telefon foarte scump, dar c...
4    detin un 13 pro max sierra 1tb din septembrie ...
Name: review_content, dtype: object

In [6]:
#sterge tag-urile html
data = data.apply(lambda x: remove_tags(x))
data.head()

C:\Users\User\anaconda3\envs\Tensorflow\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "..........." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


0        wow! fără cuvinte… pur și simplu excepțional.
1    incep prin a spune ca am precomandat telefonul...
2            sunt extraordinar de mulțumit de telefon.
3    intradevar este un telefon foarte scump, dar c...
4    detin un 13 pro max sierra 1tb din septembrie ...
Name: review_content, dtype: object

In [7]:
#sterge punctuatia
data = data.apply(lambda x: remove_punctuation(x))
data.head()

0          wow fără cuvinte pur și simplu excepțional 
1    incep prin a spune ca am precomandat telefonul...
2            sunt extraordinar de mulțumit de telefon 
3    intradevar este un telefon foarte scump dar cr...
4    detin un 13 pro max sierra 1tb din septembrie ...
Name: review_content, dtype: object

In [9]:
#elimina paranteze
data = data.apply(lambda x: remove_between_square_brackets(x))

In [10]:
#elimina stopwords
data= data.apply(lambda x: remove_stopwords(x))
data.head()

0                   wow cuvinte pur simplu excepțional
1    incep spune precomandat telefonul data 15 noie...
2                        extraordinar mulțumit telefon
3           intradevar telefon scump cred merita banii
4    detin 13 pro max sierra 1tb septembrie printre...
Name: review_content, dtype: object

In [11]:
#elimina numere
data = data.apply(lambda x: remove_numbers(x))
data.head()

0                   wow cuvinte pur simplu excepțional
1    incep spune precomandat telefonul data  noiemb...
2                        extraordinar mulțumit telefon
3           intradevar telefon scump cred merita banii
4    detin  pro max sierra tb septembrie printre pr...
Name: review_content, dtype: object

In [12]:
#eleimina literele singure
data = data.apply(lambda x: remove_single_letter(x))
data.head()

0                   wow cuvinte pur simplu excepțional
1    incep spune precomandat telefonul data noiembr...
2                        extraordinar mulțumit telefon
3           intradevar telefon scump cred merita banii
4    detin pro max sierra tb septembrie printre pri...
Name: review_content, dtype: object

In [13]:
#elimina diacriticele
data = data.apply(lambda x: remove_accents(x))
data.head()

0                   wow cuvinte pur simplu exceptional
1    incep spune precomandat telefonul data noiembr...
2                        extraordinar multumit telefon
3           intradevar telefon scump cred merita banii
4    detin pro max sierra tb septembrie printre pri...
Name: review_content, dtype: object

In [14]:
#elimina caracterele speciale
data = data.apply(lambda x: remove_special_characters(x))

In [15]:
data

0                       wow cuvinte pur simplu exceptional
1        incep spune precomandat telefonul data noiembr...
2                            extraordinar multumit telefon
3               intradevar telefon scump cred merita banii
4        detin pro max sierra tb septembrie printre pri...
                               ...                        
41923    telefon bun misca bine partea gaming fotografi...
41924    pacat telefon frumos incinge tare aplicatie in...
41925                                          telefon bun
41926            elegant fiabil frumos functional eficient
41927    pro ecran luminos durata bateriei excelenta su...
Name: review_content, Length: 41927, dtype: object

## Construirea unui model word2vec folosind stemming ca metoda de normalizare a textului, fara diacritice:


### 1. Construirea Word2vec model

In [16]:
# aplicarea stemming
s_data= data.apply(lambda x: nltk.word_tokenize(x))
s_data = s_data.apply(lambda x: [stemmer.stem(y) for y in x])
s_data

0                  [wow, cuvint, pur, simplu, exceptional]
1        [incep, spun, precomand, telefon, dat, noiembr...
2                          [extraordinar, multum, telefon]
3           [intradevar, telefon, scump, cred, merit, ban]
4        [detin, pro, max, sierr, tb, septembr, printr,...
                               ...                        
41923    [telefon, bun, misc, bin, part, gaming, fotogr...
41924    [pacat, telefon, frumos, incing, tar, aplicat,...
41925                                       [telefon, bun]
41926         [eleg, fiabil, frumos, functional, eficient]
41927    [pro, ecran, lumin, durat, bater, excelent, su...
Name: review_content, Length: 41927, dtype: object

In [17]:
##### creare model Word2Vec #####

vector_size = 300 #numar dimensiuni embedding
window = 5 #5 cuvinte inainte si 5 cuvinte dupa fiecare cuvant avut in vedere
min_count = 1 #cuvantul trebuie sa apara cel putin o data in propozitie
workers = 3 #numarul de partitii pe durata training-ului- cat de multe cpu utilizeaza
sg = 0 # algoritmul de antrenare -> 0 pentru cbow, 1 pentru skip gram

word2vec_model = Word2Vec(min_count = min_count,
                          vector_size= vector_size,
                          workers = workers, 
                          window = window, 
                          sg = sg)
# construirea vocabularului
word2vec_model.build_vocab(s_data)

In [18]:
word2vec_model.corpus_count

41927

### 2. Antrenarea modelului

In [19]:
word2vec_model.train(s_data,
                      total_examples = word2vec_model.corpus_count, 
                      epochs = word2vec_model.epochs)

(3762187, 4528520)

In [21]:
word2vec_model.save('Word2Vec_Trained_Model/w2v_stemmed_model')

In [23]:
word2vec_model = models.KeyedVectors.load('Word2Vec_Trained_Model/w2v_stemmed_model')
word2vec_model.wv.most_similar(positive='browsing', topn=5)

[('social', 0.9740762114524841),
 ('socializ', 0.966336727142334),
 ('medi', 0.9565662145614624),
 ('fb', 0.9555067420005798),
 ('instagr', 0.9343439936637878)]

In [24]:
len(word2vec_model.wv.index_to_key)

18983

In [25]:
#pentru vizualizare in tensorboard
m = models.KeyedVectors.load('Word2Vec_Trained_Model/w2v_stemmed_model')
import io
weights = m.wv.vectors
vocab = m.wv.index_to_key

out_v = io.open('Tensorboard Data/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('Tensorboard Data/metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
        vec = weights[index]
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_m.write(word + "\n")
out_v.close()
out_m.close()